In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
with ZipFile("drive/MyDrive/archive.zip",'r') as zipObj:
  zipObj.extractall('sample_data')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import keras

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg19 import VGG19, preprocess_input,decode_predictions

In [ ]:
len(os.listdir('/content/sample_data/archive/train'))

29

In [ ]:
train_datagen=ImageDataGenerator(zoom_range=0.5,shear_range=0.3,horizontal_flip=True,preprocessing_function=preprocess_input)

val_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train=train_datagen.flow_from_directory("/content/sample_data/archive/train", target_size=(256,256), batch_size=32)

val=train_datagen.flow_from_directory("/content/sample_data/archive/test", target_size=(256,256), batch_size=32)


Found 40185 images belonging to 29 classes.
Found 13392 images belonging to 29 classes.


In [ ]:
t_imag , label= train.next()

In [ ]:
t_imag.shape

(32, 256, 256, 3)

In [ ]:
def plotImage(img_arr, label):

    for im, l in zip(img_arr, label):
        plt.figure(figsize=(5,5))
        #plt.imshow(im/255)
        plt.show()

In [ ]:
plotImage(t_imag[:3],label[:3])

<Figure size 360x360 with 0 Axes>

<Figure size 360x360 with 0 Axes>

<Figure size 360x360 with 0 Axes>

In [ ]:
#Building our Model

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19, preprocess_input,decode_predictions
import keras

In [ ]:
base_model=VGG19(input_shape=(256,256,3),include_top=False)

80150528/80134624 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
x=Flatten()(base_model.output)

x=Dense(units=29,activation='softmax')(x)


#Creating model
model=Model(base_model.input,x)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

Early Stopping and Model Check Point

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# early stopping
es=EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=3,verbose=1)

# model check point
mc= ModelCheckpoint(filepath="best_model.h5",monitor='val_accuracy',min_delta=0.01,patience=3,verbose=1,save_best_only=True)

cb=[es,mc]

In [ ]:
his=model.fit_generator(train, steps_per_epoch=16, epochs=50, verbose=1, callbacks=cb, validation_data=val, validation_steps=16)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/50
16/16 [==============================] - ETA: 0s - loss: 23.0858 - accuracy: 0.2227 
Epoch 1: val_accuracy improved from -inf to 0.33594, saving model to best_model.h5
16/16 [==============================] - 769s 50s/step - loss: 23.0858 - accuracy: 0.2227 - val_loss: 19.3982 - val_accuracy: 0.3359
Epoch 2/50
 6/16 [==========>...................] - ETA: 3:56 - loss: 13.7556 - accuracy: 0.3594

In [ ]:
h=his.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'],c='red')
plt.title('acc VS v-acc')
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_accuracy'],c='red')
plt.title('loss VS v-acc')
plt.show()

In [ ]:
# load best model

from keras.models import load_model

model=load_model("best_model.h5")

In [ ]:
acc=model.evaluate_generator(val)[1]

print(f"The accuracy of your model is {acc*100} %")

In [ ]:
ref = dict(zip(list(train.class_indices.values()),list(train.class_indices.keys())))

In [ ]:
import pickle

with open("myDict.txt", "wb") as myFile:
    pickle.dump(ref, myFile)

In [ ]:
def prediction(path):
    img=load_img(path,target_size=(256,256))

    i=img_to_array(img)

    im=preprocess_input(i)

    # print(im)

    img=np.expand_dims(im, axis=0)

    pred=np.argmax(model.predict(img))
    
    print(pred)

    print(f"the image belongs to {ref [pred]}")

In [ ]:
path='/content/powdery.jpg'

prediction(path)